In [1]:
import pandas as pd
import numpy as np

In [42]:
# load data
path = "./data/"
donation = pd.read_csv(path+"donations.csv")
email = pd.read_csv(path+"emails.csv")
year_joined = pd.read_csv(path+"year_joined.csv")

email["emailsOpened"] = email["emailsOpened"].astype("int")
email["user"] = email["user"].astype("int")
donation["user"] = donation["user"].astype("int")
email["week"] = pd.to_datetime(email['week'], format='%Y-%m-%d %H:%M:%S')
donation["timestamp"] = pd.to_datetime(donation['timestamp'], format='%Y-%m-%d %H:%M:%S')
year_joined["yearJoined"] = pd.to_datetime(year_joined['yearJoined'], format='%Y')

In [43]:
year_joined.head(3)

,user,userStats,yearJoined
0,0,silver,2014-01-01
1,1,silver,2015-01-01
2,2,silver,2016-01-01


In [44]:
email.head(3)

,emailsOpened,user,week
0,3,1,2015-06-29
1,2,1,2015-07-13
2,2,1,2015-07-20


In [45]:
donation.head(3)

,amount,timestamp,user
0,25.0,2017-11-12 11:13:44,0
1,50.0,2015-08-25 19:01:45,0
2,25.0,2015-03-26 12:03:47,0


In [46]:
# 複数の会員資格を持つユーザーを検索
# year_joined.groupby("user").count() : ユーザーIDでGROUPBYしてユーザーID数をカウント
# *.groupby("userStats").count() : userStatsでGROUPBYして資格の数をカウント
year_joined.groupby("user").count().groupby("userStats").count()

,yearJoined
userStats,
1,1000


次のコードでデータを追加して実行すると, 会員資格が複数あるユーザーがいるか検索できる

```
year_joined.loc[1000] = [0,"gold",2021]
```

```
year_joined.groupby("user").count().groupby("userStats").count()

userStats yearJoined	
1	999
2	1
```

In [47]:
# 1件もメールを開封していないユーザーがいるか
email[email.emailsOpened<1]

,emailsOpened,user,week


実行結果からメールを1件も開けていないユーザーを記録していないか, 少なくとも1回はメールが開封されていることがわかる.  
全ユーザーが週最低1回はメールを開くか? -> そのような仮説は正しくなさそう.

In [48]:
# とあるユーザーの記録を見る
email[email.user==998]

,emailsOpened,user,week
25464,1,998,2017-12-04
25465,3,998,2017-12-11
25466,3,998,2017-12-18
25467,3,998,2018-01-01
25468,3,998,2018-01-08
25469,2,998,2018-01-15
25470,3,998,2018-01-22
25471,2,998,2018-01-29
25472,3,998,2018-02-05
25473,3,998,2018-02-12


年末にメールを開いた形跡がないことから開いていないときは記録されていないと考えられる.

In [49]:
# 会員番号998の在会期間を週ごとで計算
(max(email[email.user==998].week) - min(email[email.user==998].week)).days/7

25.0

この計算では結果は25であるが, 週は26である. 例えば4/7,14,21,28のデータがあるとき(最大の週-最小の週)/7は3になる. しかし週の数は4である. このため1を足す必要がある.

In [52]:
# データの週の数を確認
email[email.user==998].shape

(24, 3)

本来の週の数は26だが, 24週しかないため欠けている週があることがわかる.

In [53]:
# 週と会員のすべての組み合わせのインデックスを生成
complete_idx = pd.MultiIndex.from_product((set(email.week),set(email.user)))

In [54]:
# 欠けている週を埋める処理
all_email = email.set_index(["week","user"]).reindex(complete_idx,fill_value=0).reset_index()
all_email.columns = ["week","user","emailsOpened"]

In [55]:
all_email[all_email.user==998].sort_values("week")

,week,user,emailsOpened
69530,2015-02-09,998,0
36112,2015-02-16,998,0
53360,2015-02-23,998,0
67913,2015-03-02,998,0
16708,2015-03-09,998,0
...,...,...,...
33417,2018-04-30,998,3
91629,2018-05-07,998,3
63062,2018-05-14,998,3
11318,2018-05-21,998,3


初めてメールが開封されるまでは会員登録していないことが考えられる. そこで初めてメールを開封するまでのデータはカットする.

In [62]:
# 会員ごとに最初と最後に既読した週を抽出
# agg : まとめてGROPUBYする関数

cutoff_dates = email.groupby("user").week.agg(["min","max"]).reset_index()
cutoff_dates

,user,min,max
0,1,2015-06-29,2018-05-28
1,3,2018-03-05,2018-04-23
2,5,2017-06-05,2018-05-28
3,6,2016-12-05,2018-05-28
4,9,2016-07-18,2018-05-28
...,...,...,...
534,991,2016-10-24,2016-10-24
535,992,2015-02-09,2015-07-06
536,993,2017-09-11,2018-05-28
537,995,2016-09-05,2018-05-28


In [63]:
# 最初の非ゼロ数以前の行と最後の非ゼロ数以後の行を削除

for _,row in cutoff_dates.iterrows():
    user=row["user"]
    start_date = row["min"]
    end_date = row["max"]
    all_email.drop(all_email[all_email.user==user][all_email.week<start_date].index, inplace=True)
    all_email.drop(all_email[all_email.user==user][all_email.week>end_date].index, inplace=True)

<ipython-input-63-b97588936713>:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  all_email.drop(all_email[all_email.user==user][all_email.week<start_date].index, inplace=True)
<ipython-input-63-b97588936713>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  all_email.drop(all_email[all_email.user==user][all_email.week>end_date].index, inplace=True)


In [65]:
all_email[all_email.user==998]

,week,user,emailsOpened
3233,2017-12-18,998,3
11318,2018-05-21,998,3
11857,2017-12-11,998,3
14013,2018-01-08,998,3
19403,2018-05-28,998,3
22637,2018-02-05,998,3
25332,2018-03-12,998,3
29644,2018-04-23,998,0
30722,2018-01-01,998,3
33417,2018-04-30,998,3
